In [1]:
import pandas as pd

# Pandas

In [7]:
import pandas as pd

def calculate_mape_by_category(df, actual_col, prediction_col, category_col):
    """
    Calculate MAPE for each category in the DataFrame, handling zero actual values.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    actual_col (str): The column name for actual values.
    prediction_col (str): The column name for predicted values.
    category_col (str): The column name for categories.

    Returns:
    pd.DataFrame: A DataFrame with categories and their corresponding MAPE values.
    """
    
    # Function to calculate MAPE for a single category, excluding zero actual values
    def mape(actual, predicted):
        mask = actual != 0
        return (abs((actual[mask] - predicted[mask]) / actual[mask])).mean() * 100
    
    # Group by category and calculate MAPE for each group
    mape_results = df.groupby(category_col).apply(
        lambda group: mape(group[actual_col], group[prediction_col])
    ).reset_index(name='MAPE')
    
    return mape_results

# Example usage:
# Assuming df is your DataFrame with columns 'A', 'B', 'C', and 'D'
# df = pd.DataFrame({
#     'A': [1, 2, 3, 4, 5, 6],
#     'B': ['cat1', 'cat2', 'cat1', 'cat2', 'cat1', 'cat2'],
#     'C': [100, 0, 150, 230, 0, 220],
#     'D': [110, 210, 140, 250, 170, 210]
# })

# Calculate MAPE by category
# mape_by_category = calculate_mape_by_category(df, actual_col='C', prediction_col='D', category_col='B')
# print


In [8]:
# Create a sample DataFrame
data = {
    'A': [1, 2, 3, 4, 5, 6],
    'B': ['cat1', 'cat2', 'cat1', 'cat2', 'cat1', 'cat2'],
    'C': [100, 0, 150, 230, 0, 220],
    'D': [110, 210, 140, 250, 170, 210]
}
df = pd.DataFrame(data)

In [10]:
print(type(df))
df

<class 'pandas.core.frame.DataFrame'>


,A,B,C,D
0,1,cat1,100,110
1,2,cat2,0,210
2,3,cat1,150,140
3,4,cat2,230,250
4,5,cat1,0,170
5,6,cat2,220,210


In [11]:
# Calculate MAPE by category
mape_by_category = calculate_mape_by_category(df, actual_col='C', prediction_col='D', category_col='B')
print(mape_by_category)
print(type(mape_by_category))

      B      MAPE
0  cat1  8.333333
1  cat2  6.620553
<class 'pandas.core.frame.DataFrame'>


# PySpark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, abs as spark_abs, mean

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("MAPE Calculation").getOrCreate()

In [ ]:
def calculate_mape_by_category(df, actual_col, prediction_col, category_col):
    """
    Calculate MAPE for each category in the PySpark DataFrame, handling zero actual values.

    Parameters:
    df (pyspark.sql.DataFrame): The DataFrame containing the data.
    actual_col (str): The column name for actual values.
    prediction_col (str): The column name for predicted values.
    category_col (str): The column name for categories.

    Returns:
    pyspark.sql.DataFrame: A DataFrame with categories and their corresponding MAPE values.
    """
    
    # Create a column for absolute percentage error, handling zero actual values
    df = df.withColumn(
        'absolute_percentage_error',
        when(col(actual_col) != 0, spark_abs((col(actual_col) - col(prediction_col)) / col(actual_col)) * 100).otherwise(None)
    )
    
    # Calculate the mean absolute percentage error for each category
    mape_results = df.groupBy(category_col).agg(mean(col('absolute_percentage_error')).alias('MAPE'))
    
    return mape_results

# Example usage:
# Assuming df is your PySpark DataFrame with columns 'A', 'B', 'C', and 'D'
# data = [
#     (1, 'cat1', 100, 110),
#     (2, 'cat2', 0, 210),
#     (3, 'cat1', 150, 140),
#     (4, 'cat2', 230, 250),
#     (5, 'cat1', 0, 170),
#     (6, 'cat2', 220, 210)
# ]
# columns = ['A', 'B', 'C', 'D']
# df = spark.createDataFrame(data, columns)

# Calculate MAPE by category
# mape_by_category = calculate_mape_by_category(df, actual_col='C', prediction_col='D', category_col='B')
# mape_by_category.show()


In [ ]:
data2 = [
    (1, 'cat1', 100, 110),
    (2, 'cat2', 0, 210),
    (3, 'cat1', 150, 140),
    (4, 'cat2', 230, 250),
    (5, 'cat1', 0, 170),
    (6, 'cat2', 220, 210)
]
columns2 = ['A', 'B', 'C', 'D']
df2 = spark.createDataFrame(data2, columns2)


In [ ]:
# Calculate MAPE by category
mape_by_category = calculate_mape_by_category(df, actual_col='C', prediction_col='D', category_col='B')
mape_by_category.show()